In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

import os
import gc

from sklearn.model_selection import train_test_split


import tensorflow as tf
from tqdm.autonotebook import tqdm

import numpy as np #
import pandas as pd 

from keras import Sequential
from keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam,SGD
from keras.callbacks import ReduceLROnPlateau
from keras.layers import Flatten,Dense,BatchNormalization,Activation,Dropout
from keras.layers import Lambda, Input, GlobalAveragePooling2D,BatchNormalization
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Model
from keras.preprocessing.image import load_img

In [ ]:
labels = pd.read_csv('../input/dog-breed-identification/labels.csv')
labels.head()

In [ ]:
labels.describe()

In [ ]:
def barw(ax): 
    
    for p in ax.patches:
        val = p.get_width() 
        x = p.get_x()+ p.get_width()
        y = p.get_y() + p.get_height()/2
        ax.annotate(round(val,2),(x,y))

plt.figure(figsize = (15,30))
ax0 =sns.countplot(y=labels['breed'],order=labels['breed'].value_counts().index)
barw(ax0)
plt.show()

In [ ]:
import os
if len(os.listdir('../input/dog-breed-identification/train')) == len(labels['id']):
    print('equal images and labels')
else:
    print('number of images and labels are not equal')

In [ ]:
classes = sorted(list(set(labels['breed'])))
n_classes = len(classes)
print('Total breeds {}'.format(n_classes))
class_to_num = dict(zip(classes, range(n_classes)))
class_to_num

In [ ]:
input_shape = (331,331,3)


def images_to_array(directory, label_dataframe, target_size = input_shape):
    
    image_labels = label_dataframe['breed']
    images = np.zeros([len(label_dataframe), target_size[0], target_size[1], target_size[2]],dtype=np.uint8) #as we have huge data and limited ram memory. uint8 takes less memory
    y = np.zeros([len(label_dataframe),1],dtype = np.uint8)
    
    for ix, image_name in enumerate(tqdm(label_dataframe['id'].values)):
        img_dir = os.path.join(directory, image_name + '.jpg')
        img = load_img(img_dir, target_size = target_size)
#         img = np.expand_dims(img, axis=0)
#         img = processed_image_resnet(img)
#         img = img/255
        images[ix]=img
#         images[ix] = img_to_array(img)
        del img
        
        dog_breed = image_labels[ix]
        y[ix] = class_to_num[dog_breed]
    y = to_categorical(y)
    
    return images,y

In [ ]:
X,y = images_to_array('../input/dog-breed-identification/train', labels[:])

In [ ]:
gc.collect()

In [ ]:
n=25

plt.figure(figsize=(20,20))

for i in range(n):
    ax = plt.subplot(5, 5, i+1)
    plt.title(classes[np.where(y[i] ==1)[0][0]])
    plt.imshow(X[i].astype('int32')) 

In [ ]:
lrr= ReduceLROnPlateau(monitor='val_acc', factor=.01, patience=3, min_lr=1e-5,verbose = 1)

EarlyStop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

In [ ]:
batch_size= 128
epochs=50
learn_rate=.001
sgd=SGD(lr=learn_rate,momentum=.9,nesterov=False)
adam=Adam(lr=learn_rate, beta_1=0.9, beta_2=0.999, epsilon=None,  amsgrad=False)

In [ ]:
img_size = (331,331,3)

def get_features(model_name, model_preprocessor, input_size, data):

    input_layer = Input(input_size)
    preprocessor = Lambda(model_preprocessor)(input_layer)
    base_model = model_name(weights='imagenet', include_top=False,
                            input_shape=input_size)(preprocessor)
    avg = GlobalAveragePooling2D()(base_model)
    feature_extractor = Model(inputs = input_layer, outputs = avg)
    feature_maps = feature_extractor.predict(data, verbose=1)
    print('Feature maps shape: ', feature_maps.shape)
    return feature_maps

In [ ]:
from keras.applications.inception_v3 import InceptionV3, preprocess_input
inception_preprocessor = preprocess_input
inception_features = get_features(InceptionV3,
                                  inception_preprocessor,
                                  img_size, X)

In [ ]:
from keras.applications.xception import Xception, preprocess_input
xception_preprocessor = preprocess_input
xception_features = get_features(Xception,
                                 xception_preprocessor,
                                 img_size, X)

In [ ]:
gc.collect()

In [ ]:
final_features = np.concatenate([inception_features,xception_features], axis=-1)
print('Final feature maps shape', final_features.shape)

In [ ]:
model = Sequential()
model.add(Dropout(0.7,input_shape=(final_features.shape[1],)))
model.add(Dense(n_classes,activation= 'softmax'))

model.compile(optimizer=adam,
              loss='categorical_crossentropy',
              metrics=['accuracy'])

#Training the model. 
history = model.fit(final_features, y,
            batch_size=batch_size,
            epochs=epochs,
            validation_split=0.2,
            callbacks=[lrr,EarlyStop])

In [ ]:
del inception_features
del xception_features
del final_features
gc.collect()

In [ ]:
def extact_features(data):
    inception_features = get_features(InceptionV3, inception_preprocessor, img_size, data)
    xception_features = get_features(Xception, xception_preprocessor, img_size, data)
    gc.collect()
    final_features = np.concatenate([inception_features,xception_features],axis=-1)
    
    print('Final feature maps shape', final_features.shape)

    del inception_features
    del xception_features
    gc.collect()
    
    
    return final_features



In [ ]:
#test_features = extact_features(test_data)

In [ ]:
#del test_data
gc.collect()

In [ ]:
from IPython.display import display
from PIL import Image

def predict_dog(link):
    basewidth = 300
    img = Image.open(link)
    wpercent = (basewidth/float(img.size[0]))
    hsize = int((float(img.size[1])*float(wpercent)))
    img = img.resize((basewidth,hsize), Image.ANTIALIAS)
    display(img)
    img_g = load_img(link,target_size = img_size)
    img_g = np.expand_dims(img_g, axis=0)
    img_g.shape
    test_features = extact_features(img_g)
    predg = model.predict(test_features)
    print(f"Predicted label: {classes[np.argmax(predg[0])]}")
    print(f"Probability of prediction): {round(np.max(predg[0])) * 100} %")

In [ ]:
predict_dog("../input/custom-test/greatdane.jpg")

In [ ]:
predict_dog('../input/custom-test/german_shepherd.jpeg')

In [ ]:
predict_dog('../input/custom-test/unknown_dog1.jpg')

In [ ]:
predict_dog('../input/custom-test/vit_dog1.jpeg')

In [ ]:
predict_dog('../input/custom-test/vit_dog2.jpeg')

In [ ]:
predict_dog('../input/custom-test/house_dog1.jpeg')